# Chapter 7 - Example: Exploratory Analysis of Saildrone Cruise Data

This script is an example of an exploratory analysis I performed on Saildrone data. The script reads in the data, calculates some new fields, and then generates graphs and maps to help examine the Saildrone data. The script is broken up into different parts and includes comments throughout to help you follow along. 



### Load Packages and Read in Data

In [ ]:
#import necessary packages
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
import seaborn as sns
from geopy import distance
from datetime import datetime

#load map packages
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from calendar import month_abbr

In [ ]:
#open .nc file with Saildrone Data
sail = xr.open_dataset('data/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1jplv05.0_orbital.nc') 
sail.close()

#copy data for calculations versus graphing
sail_raw = sail.copy()

sail

### Filter data and calculate a new column

In [ ]:
#filter out salinity values that are less than or equal to 32
sail['SAL_CTD_MEAN']=sail.SAL_CTD_MEAN.where(sail.SAL_CTD_MEAN>=32, np.nan)

#resample the data by hour
sail = sail.resample(time="H").mean(skipna=True)

#add date column but store it as a matplotlib date format
sail['date'] = mdates.date2num(sail['time'].dt.date)

#create function for calculating windspeed
def windspeed(u, v):
    return np.sqrt((np.abs(u**2) + np.abs(v**2)))

#calculate windspeed using function
sail['wind_speed'] = windspeed(sail['UWND_MEAN'], sail['VWND_MEAN'])


### View Windspeed throughout the Cruise

In [ ]:
#resample the data by Day
resampled_wind = sail.resample(time="D", skipna = True).mean()

#create list of ticks for the x axis
default_x_ticks = range(0,len(sail['time']),divmod(len(sail['time']), 10)[0]) #selects out 10 evenly spaced dates from the data
colors = plt.cm.jet(np.linspace(0, 1, len(resampled_wind)))

#set figure size
plt.figure(figsize = (14,4))

#create lineplot 
sns.lineplot(x = sail['time'].dt.date, y = sail['wind_speed'], color = 'grey', alpha=0.2, ci=None) #seaborn has built in confidence intervals
plt.scatter(sail['time'].dt.date, sail['wind_speed'], 40, [*range(len(sail['wind_speed']))], 'o', cmap='jet') #add points for each day
plt.tight_layout()

#add labels
plt.title('Wind Speed for Saildrone Cruise', fontdict = {'fontsize' : 16}) 
plt.xlabel("Date")
plt.ylabel("Wind Speed [m/s]")

#select x axis ticks based on above values
plt.xticks(sail['time'][default_x_ticks], sail['time'].dt.date[default_x_ticks].values)
plt.grid(zorder=0, alpha=0.5)
plt.show()

#set figure size
plt.figure(figsize = (14,4))

#create lineplot 
sns.lineplot(x = sail['time'].dt.date, y = sail['wind_speed'], color = 'grey', alpha=0.2, ci='sd') #seaborn has built in confidence intervals
plt.scatter(resampled_wind['time'].dt.date, resampled_wind['wind_speed'], 40, [*range(len(resampled_wind['wind_speed']))], 'o', cmap='jet') #add points for each day
plt.tight_layout()

#add labels
plt.title('Daily Average of Wind Speed for Saildrone Cruise', fontdict = {'fontsize' : 16}) 
plt.xlabel("Date")
plt.ylabel("Wind Speed [m/s]")

#select x axis ticks based on above values
plt.xticks(sail['time'][default_x_ticks], sail['time'].dt.date[default_x_ticks].values)
plt.grid(zorder=0, alpha=0.5)
plt.show()

### View Temperature throughout the Cruise

In [ ]:
#resample the data by Day
resampled_temp = sail.resample(time="D", skipna = True).mean()

#set figure size
plt.figure(figsize = (14,4))

#create lineplot 
sns.lineplot(x = sail['time'].dt.date, y = sail['TEMP_CTD_MEAN'], color = 'grey', ci = 'sd', alpha=0.2)
plt.scatter(resampled_temp['time'].dt.date, resampled_temp['TEMP_CTD_MEAN'], 30, [*range(len(resampled_temp['TEMP_CTD_MEAN']))], 'o', cmap='jet')
plt.tight_layout()

#set labels
plt.title('Water Temperature for Saildrone Cruise', fontdict = {'fontsize' : 16})
plt.xlabel("Date")
plt.ylabel("Temperature [C]")

#reformat x-axis ticks
plt.xticks(sail['time'][default_x_ticks], sail['time'].dt.date[default_x_ticks].values)
plt.grid(zorder=0, alpha=0.5)
plt.show()

### View Salinity throughout the Cruise

In [ ]:
#resample the data by Day
resampled_sal = sail.resample(time="D", skipna = True).mean()

#set figure
plt.figure(figsize = (14,4))

#create lineplot 
sns.lineplot(x = sail['time'].dt.date, y = sail['SAL_CTD_MEAN'], color = 'grey', ci = 'sd', alpha=0.2)
plt.scatter(resampled_sal['time'].dt.date, resampled_sal['SAL_CTD_MEAN'], 30, [*range(len(resampled_sal['SAL_CTD_MEAN']))], 'o', cmap='jet')
plt.tight_layout()

#label graph
plt.title('Salinity for Saildrone Cruise', fontdict = {'fontsize' : 16})
plt.xlabel("Date")
plt.ylabel("Salinity [PPT]")

#reformat ticks
plt.xticks(sail['time'][default_x_ticks], sail['time'].dt.date[default_x_ticks].values)
plt.grid(zorder=0, alpha=0.5)
plt.show()

### Look at Sampling Tracks

In [ ]:
#define latitude and longitude boundaries
latr = [min(sail['lat']), max(sail['lat'])] 
lonr = [max(sail['lon']), min(sail['lon'])] 

# Select a region of our data, giving it a margin
margin = 0.5 
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]+margin,lonr[1]-margin]]) 

#add state outlines
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')

# Create and set the figure context
fig = plt.figure(figsize=(16,10), dpi = 72) 
ax = plt.axes(projection=ccrs.PlateCarree()) 
ax.coastlines(resolution='10m',linewidth=1,color='black') 
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(states_provinces, linewidth = 0.5)
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
ax.set_xticks(np.round([*np.arange(region[1,1],region[1,0]+1,2)][::-1],0), crs=ccrs.PlateCarree()) 
ax.set_yticks(np.round([*np.arange(np.floor(region[0,0]),region[0,1]+1,1.5)],1), crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
ax.gridlines(linestyle = '--', linewidth = 0.5)

# Plot track data, color by temperature
loc = mdates.AutoDateLocator()
sc = plt.scatter(x = sail['lon'], y = sail['lat'], c = sail['date'], cmap='jet')
#plt.plot(sail['lon'], sail['lat'], ls = ':', c = 'navy', alpha = 0.4)
clb = fig.colorbar(sc, ticks=loc, format=mdates.AutoDateFormatter(loc))
clb.ax.set_title('Date')
clb.set_ticks(sail['date'][default_x_ticks])
clb.set_ticklabels(sail['time'].dt.date[default_x_ticks].values)
plt.title('Sampling Track for Cruise', fontdict = {'fontsize' : 16})
plt.show()

### Salinity versus Temperature Diagram

In [ ]:
#filter data
sail_filtered = sail.drop('date')
sail_filtered = sail_filtered.to_dataframe()
sail_filtered['date'] = mdates.date2num(sail['time'].dt.date)
#sail_filtered.reset_index(inplace=True)
#sail_filtered['date'] = sail_filtered['time'].dt.date
sail_filtered = sail_filtered.loc[(sail_filtered['SAL_CTD_MEAN'] >= 32) & (sail_filtered['TEMP_CTD_MEAN'] >= 6), 
                                  ['date','SAL_CTD_MEAN','TEMP_CTD_MEAN','lat','lon']]

#specify salinity values
xaxis_val = [32, 32.5, 33, 33.5, 34]

#visualize temp and salinity data
fig = plt.figure() 
ax = plt.axes() 
plt.scatter(sail_filtered['SAL_CTD_MEAN'], sail_filtered['TEMP_CTD_MEAN'], alpha = 0.3)
plt.title('Salinity versus Temperature', fontdict = {'fontsize' : 16})
plt.xlabel("Salinity [PPT]") 
plt.ylabel("Temperature [C]")
plt.xticks(xaxis_val)
plt.show()

#visualize temp and salinity data with date
fig = plt.figure(figsize=(7.6,4)) 
loc = mdates.AutoDateLocator()
sc = plt.scatter(x = sail_filtered['SAL_CTD_MEAN'], y = sail_filtered['TEMP_CTD_MEAN'], c = sail_filtered['date'], cmap='jet', alpha = 0.5)
clb = fig.colorbar(sc, ticks=loc, format=mdates.AutoDateFormatter(loc))
#ax.set_label('Date')
plt.title('Salinity versus Temperature with Date', fontdict = {'fontsize' : 16})
plt.xlabel("Salinity [PPT]")
plt.ylabel("Temperature [C]")
plt.xticks(xaxis_val)
plt.grid()
plt.show()



### Temperature Values throughout the Cruise

In [ ]:
#define latitude and longitude boundaries
latr = [min(sail['lat']), max(sail['lat'])] 
lonr = [max(sail['lon']), min(sail['lon'])] 

# Select a region of our data, giving it a margin
margin = 0.5 
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]+margin,lonr[1]-margin]]) 

#add state outlines
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')

# Create and set the figure context
fig = plt.figure(figsize=(16,10), dpi = 72) 
ax = plt.axes(projection=ccrs.PlateCarree()) 
ax.coastlines(resolution='10m',linewidth=1,color='black') 
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(states_provinces, linewidth = 0.5)
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
ax.set_xticks(np.round([*np.arange(region[1,1],region[1,0]+1,2)][::-1],0), crs=ccrs.PlateCarree()) 
ax.set_yticks(np.round([*np.arange(np.floor(region[0,0]),region[0,1]+1,1.5)],1), crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
ax.gridlines(linestyle = '--', linewidth = 0.5)

# Plot track data, color by temperature
sc = plt.scatter(data = sail_filtered, x = 'lon', y = 'lat', c = 'TEMP_CTD_MEAN')
clb = fig.colorbar(sc, label = "Temperature [C]")
plt.set_cmap('Reds')
plt.title('Raw Temperature Values', fontdict = {'fontsize' : 16})
plt.show()

### Salinity Values throughout the Cruise

In [ ]:
#define latitude and longitude boundaries
latr = [min(sail['lat']), max(sail['lat'])] 
lonr = [max(sail['lon']), min(sail['lon'])] 

# Select a region of our data, giving it a margin
margin = 0.5 
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]+margin,lonr[1]-margin]]) 

#add state outlines
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')

# Create and set the figure context
fig = plt.figure(figsize=(16,10), dpi = 72) 
ax = plt.axes(projection=ccrs.PlateCarree()) 
ax.coastlines(resolution='10m',linewidth=1,color='black') 
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(states_provinces, linewidth = 0.5)
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
ax.set_xticks(np.round([*np.arange(region[1,1],region[1,0]+1,2)][::-1],0), crs=ccrs.PlateCarree()) 
ax.set_yticks(np.round([*np.arange(region[0,0],region[0,1]+1,1)],0), crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
ax.gridlines(linestyle = '--', linewidth = 0.5)


# Plot track data, color by temperature
sc = plt.scatter(x = sail_filtered['lon'], y = sail_filtered['lat'], c = sail_filtered['SAL_CTD_MEAN'])
clb = fig.colorbar(sc, label = "Salinity [PPT]")
plt.set_cmap('Blues')
plt.title('Raw Salinity Values', fontdict = {'fontsize' : 16})
plt.show()

In [ ]:
# #convert to dataframe 
# sail_df = sail_raw.to_dataframe()

# #create empty variables
# temp = []
# sal = []
# temp_f = []
# sal_f = []

# #for loop to go through each row of the data frame
# for i in range(0, (len(sail_df-1))):
#     #extract out first row
#     value1 = sail_df.iloc[[i]]        
#     #extract out next row
#     value2 = sail_df.iloc[[i+1]]

#     ###Temperature
#     #difference in temp
#     d_temp = value2['TEMP_CTD_MEAN'].values[0] - value1['TEMP_CTD_MEAN'].values[0]

#     ###Salinity
#     #difference in salinity
#     d_sal = value2['SAL_CTD_MEAN'].values[0] - value1['SAL_CTD_MEAN'].values[0]

#     ###Calculate distance
#     #combine latitute and longitude from first value
#     loc1 = [value1['lat'].values[0], value1['lon'].values[0]]
#     #combine lat and lon for second value
#     loc2 = [value2['lat'].values[0], value2['lon'].values[0]]

#     #calculate distance in km between two points
#     dist = distance.distance(loc1, loc2).km

#     ###Changes over distance
#     #change in temperature
#     temp_diff = d_temp / dist
#     temp_diff_f = np.sqrt((d_temp/dist)**2 + (d_temp/dist)**2)

#     #change in salinity
#     sal_diff = d_sal / dist
#     sal_diff_f = np.sqrt((d_sal/dist)**2 + (d_sal/dist)**2)

#     #add to new list
#     temp.append(temp_diff)
#     sal.append(sal_diff)
#     temp_f.append(temp_diff_f)
#     sal_f.append(sal_diff_f)
        
# #add nas for last entry
# temp.append(np.nan)
# sal.append(np.nan)
# temp_f.append(np.nan)
# sal_f.append(np.nan)

# sail_df['TEMP_DIFF'] = temp
# sail_df['SAL_DIFF'] = sal
# sail_df['TEMP_DIFF_F'] = temp_f
# sail_df['SAL_DIFF_F'] = sal_f

In [ ]:
# #create function for calculating change in values over distance 
# def saildrone_diff_front(value1, value2, lat_diff, lon_diff):
#     return np.sqrt(((value2-value1)/(lat_diff))**2 + ((value2-value1)/(lon_diff))**2)

# def saildrone_diff(value1, value2, distance_val):
#     return (value2-value1)/distance_val

In [ ]:
# #convert to dataframe 
# sail_df = sail.drop('date')
# sail_df = sail_df.to_dataframe()

# #filter out NAs and values above the threshold
# sail_df = sail_df[sail_df[['TEMP_CTD_MEAN', 'SAL_CTD_MEAN', 'lat', 'lon']].notnull().all(1)]
# sail_df = sail_df.loc[(sail_df['SAL_CTD_MEAN'] >= 32) & (sail_df['TEMP_CTD_MEAN'] >= 6)]

# #add new columns offset by one row
# sail_df['TEMP_OFFSET'] = sail_df['TEMP_CTD_MEAN'].shift(-1)
# sail_df['SAL_OFFSET'] = sail_df['SAL_CTD_MEAN'].shift(-1)
# sail_df['lat_OFFSET'] = sail_df['lat'].shift(-1)
# sail_df['lon_OFFSET'] = sail_df['lon'].shift(-1)

# latitudes = []
# longitudes = []
# distances = []

# for i in range(0, (len(sail_df)-1)):
#     lat_dist = distance.distance([sail_df['lat'][i], sail_df['lon'][i]], [sail_df['lat_OFFSET'][i], sail_df['lon'][i]]).km
#     lon_dist = distance.distance([sail_df['lat'][i], sail_df['lon'][i]], [sail_df['lat'][i], sail_df['lon_OFFSET'][i]]).km
#     dist = distance.distance([sail_df['lat'][i], sail_df['lon'][i]], [sail_df['lat_OFFSET'][i], sail_df['lon_OFFSET'][i]]).km
#     latitudes.append(lat_dist)
#     longitudes.append(lon_dist)
#     distances.append(dist)

# latitudes.append(np.nan)
# longitudes.append(np.nan)
# distances.append(np.nan)

# sail_df['lat_DIFF'] = latitudes
# sail_df['lon_DIFF'] = longitudes
# sail_df['DISTANCES'] = distances
               
    
# sail_df = sail_df.loc[(sail_df['lat_DIFF'] >= 0.1) & (sail_df['lon_DIFF'] >= 0.1)]


# #dist_lat = []
# #dist_lon = []

# # for i in range(0, (len(sail_df)-1)):
# #     lat, lon = distance_calc(sail_df['lat'][i], sail_df['lon'][i], sail_df['lat_OFFSET'][i], sail_df['lon_OFFSET'][i])
# #     dist_lat.append(lat)
# #     dist_lon.append(lon)
    
# # dist_lat.append(np.nan)
# # dist_lon.append(np.nan)

# # sail_df['lat_DIFF'] = dist_lat
# # sail_df['lon_DIFF'] = dist_lon

# # #calculate the difference in temperature
# # sail_df['TEMP_DIFF'] = saildrone_diff_front(sail_df['TEMP_CTD_MEAN'], sail_df['TEMP_OFFSET'], sail_df['lat_DIFF'], sail_df['lon_DIFF'])

# # #calculate the difference in salinity
# # sail_df['SAL_DIFF'] = saildrone_diff_front(sail_df['SAL_CTD_MEAN'], sail_df['SAL_OFFSET'], sail_df['lat_DIFF'], sail_df['lon_DIFF'])
 
    
# sail_df['TEMP_DIFF'] = saildrone_diff(sail_df['TEMP_CTD_MEAN'], sail_df['TEMP_OFFSET'], sail_df['DISTANCES'])

# sail_df['SAL_DIFF'] = saildrone_diff(sail_df['SAL_CTD_MEAN'], sail_df['SAL_OFFSET'], sail_df['DISTANCES'])


# sail_df


In [ ]:
# ax = sns.scatterplot(data = sail_df, x = "SAL_DIFF", y = "TEMP_DIFF", alpha = 0.3)
# plt.title('Spatial Changes in Salinity and Temperature', fontdict = {'fontsize' : 16})
# ax.set(xlabel = "Change in Salinity [PPT/km]", ylabel = "Change in Temperature [C/km]")
# plt.show()

# ax = sns.scatterplot(data = sail_df, x = "SAL_DIFF", y = "TEMP_DIFF", alpha = 0.3, color = "green")
# plt.title('Spatial Changes in Salinity and Temperature', fontdict = {'fontsize' : 16})
# ax.set(xlabel = "Change in Salinity [PPT/km]", ylabel = "Change in Temperature [C/km]")
# ax.set(xscale="log", yscale="log")
# plt.show()

In [ ]:
# #combine  TS figures
# fig, axs = plt.subplots(1, 2, figsize=(14, 4))
# plt.subplots_adjust(wspace=0.3)

# sns.scatterplot(ax = axs[0], data = sail_filtered, x = "SAL_CTD_MEAN", y = "TEMP_CTD_MEAN", alpha = 0.3)
# axs[0].set_title('Salinity versus Temperature', fontdict = {'fontsize' : 16})
# axs[0].set(xlabel = "Salinity [PPT]", ylabel = "Temperature [C]")

# sns.scatterplot(ax = axs[1], data = sail_df, x = "SAL_DIFF", y = "TEMP_DIFF", alpha = 0.3, color = 'red')
# axs[1].set_title('Spatial Changes in Salinity and Temperature', fontdict = {'fontsize' : 16})
# axs[1].set(xlabel = "Change in Salinity [PPT/km]", ylabel = "Change in Temperature [C/km]")

# plt.savefig('temp_plot/ts_curves.png', bbox_inches = 'tight')
# plt.show()

In [ ]:
# #add date column and summarize for maps
# sail_df.reset_index(inplace=True)
# sail_df['date'] = sail_df['time'].dt.date
# sail_map = sail_df.groupby('date').max()
# sail_map

# #define latitude and longitude boundaries
# latr = [min(sail_map['lat']), max(sail_map['lat'])] 
# lonr = [max(sail_map['lon']), min(sail_map['lon'])] 

# # Select a region of our data, giving it a margin
# margin = 0.5 
# region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]+margin,lonr[1]-margin]]) 

# # Create and set the figure context
# fig = plt.figure(figsize=(16,10)) 
# ax = plt.axes(projection=ccrs.PlateCarree()) 
# ax.coastlines(resolution='10m',linewidth=1,color='black') 
# ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
# ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
# ax.set_xticks([*np.arange(region[1,0],region[1,1]+1,1)], crs=ccrs.PlateCarree()) 
# ax.set_yticks([*np.arange(region[0,0],region[0,1]+1,1)], crs=ccrs.PlateCarree()) 
# ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
# ax.yaxis.set_major_formatter(LatitudeFormatter())

# # Plot track data, color by temperature
# sc = plt.scatter(x = sail_map['lon'], y = sail_map['lat'], c = sail_map['TEMP_DIFF'], edgecolor = "grey")
# clb = fig.colorbar(sc, label = "Temperature Difference (Daily Max)")
# plt.clim(-1, 1) 
# plt.set_cmap('RdBu_r')
# plt.plot(sail_map['lon'], sail_map['lat'], ls = ':', c = 'navy', alpha = 0.2)
# plt.title('Spatial Changes in Temperature', fontdict = {'fontsize' : 16})
# plt.savefig('temp_plot/temp_diff.png', bbox_inches = 'tight')
# plt.show()

In [ ]:
# #define latitude and longitude boundaries
# latr = [min(sail_map['lat']), max(sail_map['lat'])] 
# lonr = [max(sail_map['lon']), min(sail_map['lon'])] 

# # Select a region of our data, giving it a margin
# margin = 0.5 
# region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]+margin,lonr[1]-margin]]) 

# # Create and set the figure context
# fig = plt.figure(figsize=(16,10)) 
# ax = plt.axes(projection=ccrs.PlateCarree()) 
# ax.coastlines(resolution='10m',linewidth=1,color='black') 
# ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
# ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
# ax.set_xticks([*np.arange(region[1,0],region[1,1]+1,1)], crs=ccrs.PlateCarree()) 
# ax.set_yticks([*np.arange(region[0,0],region[0,1]+1,1)], crs=ccrs.PlateCarree()) 
# ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
# ax.yaxis.set_major_formatter(LatitudeFormatter())

# # Plot track data, color by salinity
# sc = plt.scatter(x = sail_map['lon'], y = sail_map['lat'], c = sail_map['SAL_DIFF'], edgecolor = "grey")
# clb = fig.colorbar(sc, label = "Salinity Difference (Daily Max)")
# plt.plot(sail_map['lon'], sail_map['lat'], ls = ':', c = 'navy', alpha = 0.2)
# plt.set_cmap('RdBu_r')
# plt.clim(-1,1)
# plt.title('Spatial Changes in Salinity', fontdict = {'fontsize' : 16})
# plt.savefig('temp_plot/salinity_diff.png', bbox_inches = 'tight')
# plt.show()